In [1]:
containing_folder = '/home/jpmarceaux/'
import sys
sys.path.insert(0, containing_folder)
from jpq import *
import numpy as np
from PIL import Image, ImageDraw
import random

In [2]:
def embedded_1qubit(label, width):
    """channel of embedded 1-qubit gate in circuit of given width"""
    tstr = label[0]
    idx = label[1]
    if tstr == 'x':
        g = SigmaX()
    elif tstr == 'y':
        g = SigmaY()
    elif tstr == 'z':
        g = SigmaZ()
    elif tstr == 'h':
        g = (1/np.sqrt(2))*np.array([[1, 1], [1, -1]])
    elif tstr == 's':
        g = np.array([[1, 0], [0, 1j]])
    elif tstr == 't':
        g = np.array([[1, 0], [0, (1+1j)/np.sqrt(2)]])
    else:
        raise ValueError(f"tchar {tstr} not in dictionary")
    unitary = 1
    for w in range(width):
        if w == idx:
            unitary = np.kron(unitary, g)
        else:
            unitary = np.kron(unitary, np.eye(2, dtype=complex))
    return unitary

In [3]:
def embedded_controlledU(cbit, tbit, width, U):
    """
    make unitary operation for C_U 
    [|0><0| o ... o 1] + [|1><1| o ... o U] 
    """
    assert U.shape[0] == 2 and U.shape[1] == 2, \
        "makeCU: dimension mismatch"
    I = np.eye(2, dtype=complex)
    
    m0 = 1
    m1 = 1
    for i in range(width):
        if i == cbit:
            m0 = np.kron(m0, np.array([[1, 0], [0, 0]]))
            m1 = np.kron(m1, np.array([[0, 0], [0, 1]]))
        elif i == tbit:
            m0 = np.kron(m0, I)
            m1 = np.kron(m1, U)
        else:
            m0 = np.kron(m0, I)
            m1 = np.kron(m1, I)
    return m0 + m1

In [4]:
def label2channel(label, width):
    if(label[0][0] == 'c'):
        # controlled operation
        op = label[0]
        if(op == 'cx'):
            g = SigmaX()
        elif(op == 'cy'):
            g = SigmaY()
        elif(op == 'cz'):
            g = SigmaZ()
        else:
            raise ValueError(f"tchar {op} not in dictionary")
        U = embedded_controlledU(label[1], label[2], width, g)
    else:
        # 1-qubit unitary assumed
        U = embedded_1qubit(label, width)
    return Channel([U], rep='kraus')

In [5]:
def layer2channel(layer, width):
    if len(layer) > 0: 
        chnl = label2channel(layer[0], width)
        if(len(layer) > 1):
            for c in layer[1:]:
                chnl = compose(label2channel(c, width), chnl)
        return chnl

In [6]:
class Circuit():
    def __init__(self, width, layers=None):
        self._width = width
        self._layers = layers
    def get_layer_channel(self, i):
        """Get the ith layer channel class"""
        if (i > len(self._layers) - 1):
            raise ValueError("requested layer exceeds numbers of layers in circuit")
        return(layer2channel(self._layers[i], self._width))
    def get_width(self):
        return self._width
    def get_depth(self):
        return len(self._layers)
    def get_layer(self, idx):
        return self._layers[idx]

In [7]:
def paint_X(image, bit, layer_idx, tileDims, gateRatio):
    """For now, let's just paint a black box
    we'll specialize this function to paint specific gates
    in our gate set
    """
    draw = ImageDraw.Draw(image)
    dimX = tileDims[0]
    dimY = tileDims[1]
    dims = [((layer_idx+1-gateRatio)*dimX, (bit+0.5-gateRatio)*dimY), 
            ((layer_idx+1+gateRatio)*dimX, (bit+0.5+gateRatio)*dimY)]
    draw.rectangle(dims, fill='black')

def paint_controlledU(image, cbit, tbit, layer_idx, tileDims, gateRatio):
    """
    just paints a black box and circle for now
    """
    draw = ImageDraw.Draw(image)
    dimX = tileDims[0]
    dimY = tileDims[1]
    dims = [((layer_idx+1-gateRatio)*dimX, (tbit+0.5-gateRatio)*dimY), 
            ((layer_idx+1+gateRatio)*dimX, (tbit+0.5+gateRatio)*dimY)]
    draw.rectangle(dims, fill='black')
    dims = [((layer_idx+1-gateRatio)*dimX, (cbit+0.5-gateRatio)*dimY), 
            ((layer_idx+1+gateRatio)*dimX, (cbit+0.5+gateRatio)*dimY)]
    draw.ellipse(dims, fill='black')
    
def paint_gates(image, layer, idx, tileDims, gateRatio=1/4):
    """Paint the gates"""
    for lbl in layer:
        # this will become a switch-type interface 
        if lbl[0][0] == 'c':
            paint_controlledU(image, lbl[1], lbl[2], idx, tileDims, gateRatio)
        else:
            paint_X(image, lbl[1], idx, tileDims, gateRatio)

In [8]:
def local_bloch(state, subsystem, qubits):
    dims = [2 for _ in range(qubits)]
    traced = [j for j in range(qubits) if j != subsystem]
    rho = TrX(state, traced, dims)
    return [np.trace(rho@SigmaX()), np.trace(rho@SigmaY()), np.trace(rho@SigmaZ())]

class Painter():
    def __init__(self, tileDims=(50,50), wireRatio=1/4):
        self._tileDims = tileDims
        self._wireRatio = wireRatio
    def paint(self, circ, filename="image_out.png"):
        """Paint the circuit"""
        # imput params
        qbits = circ.get_width()
        depth = circ.get_depth()
        tileX = self._tileDims[0]
        tileY = self._tileDims[1]
        wireRatio = self._wireRatio
        rho = Register(qbits).mat()
        
        # [1] Allocate image 
        image = Image.new("RGBA", (tileX*(depth+2), tileY*qbits), (0, 0, 0, 0))
        draw = ImageDraw.Draw(image)
        
        # paint initial layer of tiles
        for q in range(qbits):
            xyz = local_bloch(rho, q, qbits)
            
            r = int(255*(1+xyz[0])/2)
            g = int(255*(1+xyz[1])/2)
            b = int(255*(1+xyz[2])/2)
            xi = int(0)
            yi = int((q+0.5)*tileY)
            xf = int(tileX)
            yf = int(yi)
            points = [(xi, yi), (xf, yf)]
            draw.line(points, width=int(tileY*wireRatio), fill=(r, g, b))
        
        # painter circuit layers
        for i in range(depth):
            channel = circ.get_layer_channel(i)
            rho = channel.apply(rho)
            for q in range(qbits):
                xyz = local_bloch(rho, q, qbits)
                r = int(255*(1+xyz[0])/2)
                g = int(255*(1+xyz[1])/2)
                b = int(255*(1+xyz[2])/2)
                xi = int((i+1)*tileX)
                yi = int((q+0.5)*tileY)
                xf = int((i+2)*tileX)
                yf = int(yi)
                points = [(xi, yi), (xf, yf)]
                draw.line(points, width=int(tileY*wireRatio), fill=(r, g, b))
            layer = circ.get_layer(i)
            paint_gates(image, layer, i, (tileX, tileY))
        
        # paint measurements
        
        image.save(filename)

In [9]:
def random_circuit_XYZHSTCnot_1D(depth, qubits):
    layers = []
    for l in range(depth):
        i = random.randint(1, 7)
        if i == 1: # X-type
            j = random.randint(0, qubits-1)
            layers.append([('x', j)])
        elif i == 2: # Y-type
            j = random.randint(0, qubits-1)
            layers.append([('y', j)])
        elif i == 3: # Z-type
            j = random.randint(0, qubits-1)
            layers.append([('z', j)])
        elif i == 4: # H-type
            j = random.randint(0, qubits-1)
            layers.append([('h', j)])
        elif i == 5: # S-type
            j = random.randint(0, qubits-1)
            layers.append([('s', j)])
        elif i == 6: # T-type
            j = random.randint(0, qubits-1)
            layers.append([('t', j)])
        elif i == 7: # Cnot-type (only neighbors)
            j = random.randint(0, qubits-1)
            k = random.randint(0, qubits-1)
            while k == j:
                k = random.randint(0, qubits-1)
            layers.append([('cx', i, j)])
                
    return layers

In [14]:
layers = random_circuit_XYZHSTCnot_1D(7,4)
print(layers)
circ = Circuit(5, layers)
painter = Painter()
painter.paint(circ)

[[('cx', 7, 2)], [('t', 2)], [('h', 0)], [('x', 1)], [('cx', 7, 3)], [('x', 1)], [('s', 0)]]


<ipython-input-8-5219a8930990>:29: ComplexWarning: Casting complex values to real discards the imaginary part
  r = int(255*(1+xyz[0])/2)
<ipython-input-8-5219a8930990>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  g = int(255*(1+xyz[1])/2)
<ipython-input-8-5219a8930990>:31: ComplexWarning: Casting complex values to real discards the imaginary part
  b = int(255*(1+xyz[2])/2)
<ipython-input-8-5219a8930990>:45: ComplexWarning: Casting complex values to real discards the imaginary part
  r = int(255*(1+xyz[0])/2)
<ipython-input-8-5219a8930990>:46: ComplexWarning: Casting complex values to real discards the imaginary part
  g = int(255*(1+xyz[1])/2)
<ipython-input-8-5219a8930990>:47: ComplexWarning: Casting complex values to real discards the imaginary part
  b = int(255*(1+xyz[2])/2)


In [21]:
layers = [[('x', 0)], ('y', 1), ('h', 1), ('t', 1)]
circ = Circuit(3, layers)
painter = Painter()
painter.paint(circ)

<ipython-input-8-5219a8930990>:29: ComplexWarning: Casting complex values to real discards the imaginary part
  r = int(255*(1+xyz[0])/2)
<ipython-input-8-5219a8930990>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  g = int(255*(1+xyz[1])/2)
<ipython-input-8-5219a8930990>:31: ComplexWarning: Casting complex values to real discards the imaginary part
  b = int(255*(1+xyz[2])/2)
<ipython-input-8-5219a8930990>:45: ComplexWarning: Casting complex values to real discards the imaginary part
  r = int(255*(1+xyz[0])/2)
<ipython-input-8-5219a8930990>:46: ComplexWarning: Casting complex values to real discards the imaginary part
  g = int(255*(1+xyz[1])/2)
<ipython-input-8-5219a8930990>:47: ComplexWarning: Casting complex values to real discards the imaginary part
  b = int(255*(1+xyz[2])/2)


IndexError: string index out of range

In [22]:
C5bitCode = [
    [('h', 0)], 
    [('s', 0)], 
    [('cz', 0, 1)],
    [('cz', 0, 3)],
    [('cy', 0, 4)],
    [('h', 1)],
    [('cz', 1, 2)],
    [('cz', 1, 3)],
    [('cx', 1, 4)], 
    [('h', 2)], 
    [('cz', 2, 0)], 
    [('cz', 2, 1)], 
    [('cx', 2, 4)],
    [('h', 3)], 
    [('s', 3)],
    [('cz', 3, 0)], 
    [('cz', 3, 2)], 
    [('cy', 3, 4)]
]
circ = Circuit(5, C5bitCode)
painter = Painter()
painter.paint(circ)

<ipython-input-8-5219a8930990>:29: ComplexWarning: Casting complex values to real discards the imaginary part
  r = int(255*(1+xyz[0])/2)
<ipython-input-8-5219a8930990>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  g = int(255*(1+xyz[1])/2)
<ipython-input-8-5219a8930990>:31: ComplexWarning: Casting complex values to real discards the imaginary part
  b = int(255*(1+xyz[2])/2)
<ipython-input-8-5219a8930990>:45: ComplexWarning: Casting complex values to real discards the imaginary part
  r = int(255*(1+xyz[0])/2)
<ipython-input-8-5219a8930990>:46: ComplexWarning: Casting complex values to real discards the imaginary part
  g = int(255*(1+xyz[1])/2)
<ipython-input-8-5219a8930990>:47: ComplexWarning: Casting complex values to real discards the imaginary part
  b = int(255*(1+xyz[2])/2)


## TODO

- Labels for gates
- Rotations by $\theta$ about a general axis
- Toffoli gate
- measurement coloring 
- coloring the controlled bit on a CNOT (amount of control)
- entanglement diagrams (latin square-type things in a measurement basis given by local 1-qubit states)

## 